# Install Packages and Setup Variables


In [1]:
!pip install -q openai==1.107.0 google-genai==1.36.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 951.0/951.0 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 8.0 MB/s eta 0:00:00


In [2]:
import os

# Set the "OPENAI_API_KEY" and "GOOGLE_API_KEY" in the Python environment. Will be used by OpenAI client later.

# os.environ["OPENAI_API_KEY"] = "<YOUR_OPENAI_API_KEY>"
# os.environ["GOOGLE_API_KEY"] =  "<YOUR_GOOGLE_API_KEY>"

from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('AITutorCourseKey')
os.environ["GOOGLE_API_KEY"] =  userdata.get('GoogleAPIKey')

In [3]:
# False: Generate the embedding for the dataset. (Associated cost with using OpenAI endpoint)
# True: Load the dataset that already has the embedding vectors.
load_embedding = False

# Load Dataset


## Download Dataset (JSON)


The dataset includes several articles from the TowardsAI blog, which provide an in-depth explanation of the LLaMA2 model.


In [4]:
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
!wget https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv

--2025-09-22 20:35:45--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 173646 (170K) [text/plain]
Saving to: ‘mini-llama-articles.csv’

mini-llama-articles 100%[===================>] 169.58K  --.-KB/s    in 0.02s   

2025-09-22 20:35:45 (9.98 MB/s) - ‘mini-llama-articles.csv’ saved [173646/173646]

--2025-09-22 20:35:46--  https://raw.githubusercontent.com/AlaFalaki/tutorial_notebooks/main/data/mini-llama-articles-with_embeddings.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP r

## Read File


In [5]:
# Split the input text into chunks of specified size.
def split_into_chunks(text, chunk_size=1024):
    chunks = []
    for i in range(0, len(text), chunk_size):
        chunks.append(text[i : i + chunk_size])

    return chunks

In [6]:
import csv

chunks = []

# Load the file as a CSV
with open("./mini-llama-articles.csv", mode="r", encoding="utf-8") as file:
    csv_reader = csv.reader(file)

    for idx, row in enumerate(csv_reader):
        if idx == 0:
            continue
            # Skip header row
        chunks.extend(split_into_chunks(row[1]))

In [7]:
print("number of articles:", idx)
print("number of chunks:", len(chunks))

number of articles: 14
number of chunks: 174


In [8]:
import pandas as pd

# Convert the JSON list to a Pandas Dataframe
df = pd.DataFrame(chunks, columns=["chunk"])

df.keys()

Index(['chunk'], dtype='object')

# Generate Embedding


In [9]:
from openai import OpenAI

client = OpenAI()


# Defining a function that converts a text to embedding vector using OpenAI's Ada model.
def get_embedding(text):
    try:
        # Remove newlines
        text = text.replace("\n", " ")
        res = client.embeddings.create(input=[text], model="text-embedding-3-small")

        return res.data[0].embedding

    except:
        return None

In [10]:
from tqdm.notebook import tqdm
import numpy as np

# Generate embedding
if not load_embedding:
    print("Generating embeddings...")
    embeddings = []
    for index, row in tqdm(df.iterrows()):
        # df.at[index, 'embedding'] = get_embedding( row['chunk'] )
        embeddings.append(get_embedding(row["chunk"]))

    embeddings_values = pd.Series(embeddings)
    df.insert(loc=1, column="embedding", value=embeddings_values)

# Or, load the embedding from the file.
else:
    print("Loaded the embedding file.")
    # Load the file as a CSV
    df = pd.read_csv("mini-llama-articles-with_embeddings.csv")
    # Convert embedding column to an array
    df["embedding"] = df["embedding"].apply(lambda x: np.array(eval(x)), 0)

Generating embeddings...


0it [00:00, ?it/s]

In [11]:
# df.to_csv('mini-llama-articles-with_embeddings.csv')

# User Question


In [14]:
# Define the user question, and convert it to embedding.
QUESTION = "How many parameters LLaMA2 model has?"
QUESTION_emb = get_embedding(QUESTION)

len(QUESTION_emb)

1536

# Test Cosine Similarity


Calculating the similarity of embedding representations can help us to find pieces of text that are close to each other. In the following sample you see how the Cosine Similarity metric can identify which sentence could be a possible answer for the given user question. Obviously, the unrelated answer will score lower.


In [15]:
BAD_SOURCE_emb = get_embedding("The sky is blue.")
GOOD_SOURCE_emb = get_embedding("LLaMA2 model has a total of 2B parameters.")

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# A sample that how a good piece of text can achieve high similarity score compared
# to a completely unrelated text.
print("> Bad Response Score:", cosine_similarity([QUESTION_emb], [BAD_SOURCE_emb]))
print("> Good Response Score:", cosine_similarity([QUESTION_emb], [GOOD_SOURCE_emb]))

> Bad Response Score: [[0.02582345]]
> Good Response Score: [[0.83150579]]


# Calculate Cosine Similarities


In [17]:
# The similarity between the questions and each part of the essay.
cosine_similarities = cosine_similarity([QUESTION_emb], df["embedding"].tolist())

print(cosine_similarities)

[[0.46765815 0.46909865 0.25971113 0.29372638 0.31958335 0.40157507
  0.41498464 0.45249703 0.45931876 0.12599986 0.11747328 0.01348146
  0.22598958 0.21437459 0.10120157 0.33063246 0.10741691 0.34680618
  0.16309231 0.08730893 0.34824497 0.22836925 0.19199103 0.26468711
  0.24924278 0.34823887 0.24827409 0.32756986 0.41443706 0.41332665
  0.46360043 0.38337504 0.46845942 0.35632694 0.35391015 0.30267611
  0.29933115 0.29250869 0.40031019 0.46465801 0.39469038 0.4103813
  0.44698043 0.43168134 0.3590223  0.33962232 0.51348066 0.20929638
  0.40200631 0.32831217 0.42828206 0.48265872 0.45030827 0.34253366
  0.32074699 0.42584572 0.24619718 0.18088074 0.23643047 0.34268401
  0.34366565 0.20474278 0.19748178 0.22432538 0.2110238  0.42294962
  0.26377462 0.30432878 0.3360229  0.38368839 0.23529497 0.24345379
  0.37069372 0.28017077 0.49049488 0.53041366 0.37813187 0.43767308
  0.37744437 0.39253553 0.30082644 0.41708985 0.46738292 0.45412096
  0.35134633 0.21220392 0.4261652  0.31595412 0.4

In [18]:
import numpy as np

number_of_chunks_to_retrieve = 3

# Sort the scores
highest_index = np.argmax(cosine_similarities)

# Pick the N highest scored chunks
indices = np.argsort(cosine_similarities[0])[::-1][:number_of_chunks_to_retrieve]
print(indices)

[114  75  89]


In [19]:
# Look at the highest scored retrieved pieces of text
for idx, item in enumerate(df.chunk[indices]):
    print(f"> Chunk {idx+1}")
    print(item)
    print("----")

> Chunk 1
by Meta that ventures into both the AI and academic spaces. The model aims to help researchers, scientists, and engineers advance their work in exploring AI applications. It will be released under a non-commercial license to prevent misuse, and access will be granted to academic researchers, individuals, and organizations affiliated with the government, civil society, academia, and industry research facilities on a selective case-by-case basis. The sharing of codes and weights allows other researchers to test new approaches in LLMs. The LLaMA models have a range of 7 billion to 65 billion parameters. LLaMA-65B can be compared to DeepMind's Chinchilla and Google's PaLM. Publicly available unlabeled data was used to train these models, and training smaller foundational models require less computing power and resources. LLaMA 65B and 33B have been trained on 1.4 trillion tokens in 20 different languages, and according to the Facebook Artificial Intelligence Research (FAIR) team,

# Augment the Prompt


In [21]:
from google import genai
from google.genai import types
from google.genai.types import HttpOptions

# Initialize client
client = genai.Client(api_key=userdata.get('GoogleAPIKey'))

# Use the Gemini API to answer the questions based on the retrieved pieces of text.
try:
    # System instructions for the AI assistant
    system_instruction = (
        "You are an assistant and expert in answering questions from a chunks of content. "
        "Only answer AI-related question, else say that you cannot answer this question."
    )

    # Create a user prompt with the user's question
    prompt = (
        "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
        "Please provide an informative and accurate answer to the following question based on the available context. Be concise and take your time. \nQuestion: {}\nAnswer:"
    )

    # Add the retrieved pieces of text to the prompt
    formatted_prompt = prompt.format("".join(df.chunk[indices]), QUESTION)

    response = client.models.generate_content(
        model="gemini-2.5-flash",
        contents=formatted_prompt,
        config=types.GenerateContentConfig(
            thinking_config=types.ThinkingConfig(thinking_budget=0),
            system_instruction=system_instruction,
            temperature=0.0,
        )
    )

    res = response.text

except Exception as e:
    print(f"An error occurred: {e}")

print(res)

Llama 2 is available in four different model sizes: 7 billion, 13 billion, 34 billion, and 70 billion parameters.


## How Augmenting the Prompt can address knowledge cutoff limitations and hallucinations

In [22]:
# Consider this as a retrieved chunk
# https://ai.meta.com/blog/llama-4-multimodal-intelligence (Summarized Content)

Example_chunk ="""
Meta has unveiled the **Llama 4 herd**, a new generation of open-weight, natively multimodal AI models designed to push the boundaries of performance, efficiency, and accessibility. The release includes **Llama 4 Scout** and **Llama 4 Maverick**, alongside a preview of the massive **Llama 4 Behemoth** teacher model. These models introduce advanced **mixture-of-experts (MoE) architectures**, native text–image integration, and record-breaking context lengths, establishing Llama 4 as a major leap in multimodal AI innovation.
**Llama 4 Scout** is a **17B active parameter model with 16 experts** (109B total parameters) that offers **10 million token context length**, far exceeding Llama 3’s 128K limit. Scout’s architecture leverages **interleaved attention layers (iRoPE)** and inference-time temperature scaling to achieve superior length generalization, enabling complex tasks like multi-document summarization, codebase reasoning, and long-context retrieval. Despite its smaller size, Scout surpasses prior Llama models and competitors such as Gemma 3 and Gemini 2.0 Flash-Lite in performance, all while being deployable on a **single NVIDIA H100 GPU**. Its multimodal training allows strong image grounding, multi-image reasoning, and visual question answering.
**Llama 4 Maverick**, also with **17B active parameters**, uses **128 experts** and totals **400B parameters**, alternating dense and MoE layers for inference efficiency. It rivals or exceeds larger models like **GPT-4o, Gemini 2.0 Flash, and DeepSeek v3** on benchmarks for coding, reasoning, multilinguality, and multimodal tasks. Its efficient design makes it deployable on a single **NVIDIA H100 DGX host**, balancing performance with cost-effectiveness. Maverick was refined through a revamped **post-training pipeline**—lightweight supervised fine-tuning, continuous **online reinforcement learning (RL)** with adaptive difficulty filtering, and direct preference optimization—resulting in superior reasoning, conversational fluency, and multimodal understanding. Maverick’s **chat version achieved an ELO score of 1417 on LMArena**, reflecting best-in-class general assistant capabilities.
At the top of the hierarchy, **Llama 4 Behemoth** serves as the **teacher model**, with **288B active parameters, 16 experts, and nearly 2 trillion total parameters**. It outperforms **GPT-4.5, Claude Sonnet 3.7, and Gemini 2.0 Pro** on STEM benchmarks like MATH-500 and GPQA Diamond. Behemoth’s scale demanded innovative training strategies, including pruning 95% of supervised fine-tuning data, advanced reinforcement learning with dynamically filtered prompts, and an optimized asynchronous MoE-parallelized RL infrastructure. These techniques boosted reasoning, coding, and multilingual capabilities while maintaining instruction-following reliability.
All Llama 4 models are **natively multimodal**, trained with large-scale text, image, and video data, and leverage **Meta’s MetaP technique** for reliable hyperparameter scaling. They support **200 languages**, with 10x more multilingual tokens than Llama 3, and employ **FP8 precision** for efficient training across trillions of tokens. Safety remains a priority: Meta integrates **Llama Guard**, **Prompt Guard**, and **CyberSecEval** for content protection, while **Generative Offensive Agent Testing (GOAT)** automates adversarial red-teaming. Llama 4 also significantly reduces **political and social bias**, refusing fewer prompts and responding more neutrally than Llama 3.
In sum, **Llama 4 Scout, Maverick, and Behemoth** represent a major leap in open AI research: compact yet powerful models with unmatched context length, multimodal fluency, reasoning power, and efficiency. By making Scout and Maverick openly available on **llama.com and Hugging Face**, Meta empowers developers, enterprises, and researchers worldwide to build the next generation of AI-driven experiences."""

In [23]:
QUESTION = "How many parameters does LLaMA 4 model have?"

system_instruction = (
    "You are an assistant and expert in answering questions from a chunks of content. "
    "Only answer AI-related question, else say that you cannot answer this question."
)

# Combining the context with the user's question
prompt = (
    "Read the following informations that might contain the context you require to answer the question. You can use the informations starting from the <START_OF_CONTEXT> tag and end with the <END_OF_CONTEXT> tag. Here is the content:\n\n<START_OF_CONTEXT>\n{}\n<END_OF_CONTEXT>\n\n"
    "Please provide an informative and accurate answer to the following question based on the available context. Be concise and take your time. \nQuestion: {}\nAnswer:"
)

formatted_prompt = prompt.format(Example_chunk, QUESTION)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=formatted_prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0),
        system_instruction=system_instruction,
        temperature=0.0,
    )
)

res = response.text
print(res)

The Llama 4 models have varying numbers of parameters:

*   **Llama 4 Scout:** 17B active parameters with 109B total parameters.
*   **Llama 4 Maverick:** 17B active parameters with 400B total parameters.
*   **Llama 4 Behemoth:** 288B active parameters with nearly 2 trillion total parameters.


# Without Augmentation


Test the Gemini API to answer the same question without the addition of retrieved documents. Basically, the LLM will use its knowledge to answer the question.


In [24]:
# Test without retrieved documents
QUESTION = "How many parameters does LLaMA 4 model have?"

# System instructions
system_instruction = "You are an assistant and expert in answering questions."

# Simple prompt without context
prompt = "Be concise and take your time to answer the following question. \nQuestion: {}\nAnswer:"
formatted_prompt = prompt.format(QUESTION)

response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=formatted_prompt,
    config=types.GenerateContentConfig(
        thinking_config=types.ThinkingConfig(thinking_budget=0),
        system_instruction=system_instruction,
        temperature=0.0,
    )
)

res = response.text
print(res)


There is no LLaMA 4 model. The latest publicly known LLaMA model is LLaMA 2.
